## ARMA Modeling: Model Selection

**Functions**

`sm.tsa.SARIMAX`

### Setting Our Environment 

Before we start, make sure you have the following libraries installed in your environment:
- **pandas**
- **statsmodels**

If you still haven't installed these packages in your environment, check **HT01_arma-estimation** (last class) for info on how to do it. Lets load the packages!

In [2]:
import pandas as pd
import statsmodels.tsa.api as tsa

### Loading the Data

We will continue to study the **term_premium** dataset. We will again focus on the **term** series.

Before we load some time series data, we need to check where our current directory is. To check/change our current directory, we use **os**

In [3]:
import os
os.getcwd()

'/Users/talespadilha/Documents/Oxford/Teaching/Financial Econometrics/20-21/python/python_course'

In [4]:
os.chdir('/Users/talespadilha/Documents/Oxford/Teaching/Financial Econometrics/20-21/python/python-introduction/course/autumn')
os.getcwd() 

'/Users/talespadilha/Documents/Oxford/Teaching/Financial Econometrics/20-21/python/python-introduction/course/autumn'

Once we are in the correct directory, we can load the Term Premium data you constructed with Max in class 1 (**data-dataset-construction**).

In [5]:
data = pd.read_hdf("data/term-premium.h5","term_premium")
data.head()

,TERM,GS10,GS1
DATE,,,
1953-04-01,0.47,2.83,2.36
1953-05-01,0.57,3.05,2.48
1953-06-01,0.66,3.11,2.45
1953-07-01,0.55,2.93,2.38
1953-08-01,0.67,2.95,2.28


In [6]:
term = data["TERM"]
term

DATE
1953-04-01    0.47
1953-05-01    0.57
1953-06-01    0.66
1953-07-01    0.55
1953-08-01    0.67
              ... 
2020-07-01    0.47
2020-08-01    0.52
2020-09-01    0.55
2020-10-01    0.66
2020-11-01    0.75
Freq: MS, Name: TERM, Length: 812, dtype: float64

### Exercise 68
Perform a model selection exercise on the term premium using

1. General-to-Specific
2. Specific-to-General
3. Minimizing an Information Criteria

This question asks us the apply the model selection techniques from Michaelmas to check which time series model is the "best". Here we will start by selecting the best model using information criteria (AIC and BIC).
Doing StG and GtS in time series in models is not ideal, as one should include the full specification up to the lag of the model (i.e. all three lags of the depentend variable in an AR(3)) unless on is trying to model seasonal effecs.


We will the space of possible models to be all possible models up to an ARMA (4,4). We will be using Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC).

- $ AIC = 2 k - 2 log(\hat{L})$
- $ BIC = k log (n) - 2 log(\hat{L}) $


In [7]:
ic = {}
# Looping over AR terms:
for ar in range(5):
    # Looping over MA terms:
    for ma in range(5):
        print(f"AR: {ar}, MA: {ma}")
        # Estimating the model:
        mod = tsa.SARIMAX(term, order=(ar, 0, ma), trend="c")
        res = mod.fit()
        # Getting AIC and BIC
        ic[(ar, ma)] = [res.aic, res.bic]
# Creating a df with the ic dict:
ic = pd.DataFrame(ic, index=["AIC", "BIC"]).T
ic.index = ic.index.set_names(["AR", "MA"])

AR: 0, MA: 0
AR: 0, MA: 1


/Users/talespadilha/opt/anaconda3/envs/python_mfe/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


AR: 0, MA: 2
AR: 0, MA: 3
AR: 0, MA: 4
AR: 1, MA: 0
AR: 1, MA: 1
AR: 1, MA: 2
AR: 1, MA: 3
AR: 1, MA: 4
AR: 2, MA: 0
AR: 2, MA: 1
AR: 2, MA: 2
AR: 2, MA: 3
AR: 2, MA: 4
AR: 3, MA: 0
AR: 3, MA: 1
AR: 3, MA: 2


/Users/talespadilha/opt/anaconda3/envs/python_mfe/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


AR: 3, MA: 3
AR: 3, MA: 4
AR: 4, MA: 0
AR: 4, MA: 1
AR: 4, MA: 2
AR: 4, MA: 3
AR: 4, MA: 4


In [8]:
ic

AIC          BIC
AR MA                          
0  0   2427.365154  2436.764155
   1   1451.028880  1465.127381
   2    890.205626   909.003627
   3    545.806184   569.303685
   4    382.344827   410.541830
1  0     88.835331   102.933832
   1    -15.051811     3.746190
   2    -17.770684     5.726818
   3    -17.682174    10.514828
   4    -18.859514    14.036988
2  0      9.403824    28.201826
   1    -19.423093     4.074409
   2    -19.190895     9.006107
   3    -17.407015    15.489487
   4    -17.062317    20.533685
3  0     -6.608400    16.889102
   1    -19.263898     8.933104
   2    -17.318065    15.578437
   3    -25.254711    12.341291
   4    -22.813049    19.482454
4  0     -8.186107    20.010895
   1    -17.399792    15.496710
   2    -15.492270    22.103733
   3    -22.814636    19.480867
   4    -16.906811    30.088192

Finding which models minimize AIC/BIC:

In [9]:
aic = ic.sort_values("AIC")
ar, ma = aic.index[0]
print(f"AIC selects AR {ar}, MA {ma}")

bic = ic.sort_values("BIC")
ar, ma = bic.index[0]
print(f"BIC selects AR {ar}, MA {ma}")

AIC selects AR 3, MA 3
BIC selects AR 1, MA 1


Checking how the ARMA(3,3) looks like:

In [10]:
res = tsa.SARIMAX(term, order=(3, 0, 3), trend="c").fit()
res.summary()

/Users/talespadilha/opt/anaconda3/envs/python_mfe/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/talespadilha/opt/anaconda3/envs/python_mfe/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                   TERM   No. Observations:                  812
Model:               SARIMAX(3, 0, 3)   Log Likelihood                  20.627
Date:                Wed, 30 Dec 2020   AIC                            -25.255
Time:                        08:28:09   BIC                             12.341
Sample:                    04-01-1953   HQIC                           -10.822
                         - 11-01-2020                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1434      0.038      3.766      0.000       0.069       0.218
ar.L1         -0.3495      0.046     -7.675      0.000      -0.439      -0.260
ar.L2          0.3322      0.037      9.072      0.000       0.260       0.404
ar.L3          0.8707      0.042     20.969      0.000       0.789       0.952
ma.L1          1.7058      0.045     37.959      0.000       1.618       1.794
ma.L2          1.4062      0.063     22.338      0.000       1.283       1.530
ma.L3          0.3798      0.027     14.266      0.000       0.328       0.432
sigma2         0.0551      0.001     39.311      0.000       0.052       0.058
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):              3351.92
Prob(Q):                              0.92   Prob(JB):                         0.00
Heteroskedasticity (H):               0.69   Skew:                             0.61
Prob(H) (two-sided):                  0.00   Kurtosis:                        12.88
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Checking how the ARMA(1,1) looks like:

In [11]:
res = tsa.SARIMAX(term, order=(1, 0, 1), trend="c").fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                   TERM   No. Observations:                  812
Model:               SARIMAX(1, 0, 1)   Log Likelihood                  11.526
Date:                Wed, 30 Dec 2020   AIC                            -15.052
Time:                        08:28:11   BIC                              3.746
Sample:                    04-01-1953   HQIC                            -7.836
                         - 11-01-2020                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0475      0.012      3.949      0.000       0.024       0.071
ar.L1          0.9498      0.008    116.642      0.000       0.934       0.966
ma.L1          0.4149      0.012     33.614      0.000       0.391       0.439
sigma2         0.0567      0.001     42.834      0.000       0.054       0.059
===================================================================================
Ljung-Box (L1) (Q):                   0.61   Jarque-Bera (JB):              3780.24
Prob(Q):                              0.43   Prob(JB):                         0.00
Heteroskedasticity (H):               0.73   Skew:                             0.61
Prob(H) (two-sided):                  0.01   Kurtosis:                        13.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### General-To-Specific Approach
We will now start with an ARMA(4, 4) and proceed with an GtS approach by looking at the t-values of the estimated parameters:

In [12]:
res = tsa.SARIMAX(term, order=(4, 0, 4), trend="c").fit()
res.tvalues

intercept     3.137549
ar.L1         2.737672
ar.L2        -1.551391
ar.L3         1.402176
ar.L4         3.372736
ma.L1         5.257345
ma.L2         5.630009
ma.L3         4.298260
ma.L4         1.397100
sigma2       36.959446
dtype: float64

In [14]:
gts_res = tsa.SARIMAX(term, order=(4, 0, 3), trend="c").fit()
gts_res.tvalues

/Users/talespadilha/opt/anaconda3/envs/python_mfe/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/talespadilha/opt/anaconda3/envs/python_mfe/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


intercept     3.573886
ar.L1        -2.546671
ar.L2         1.003957
ar.L3        13.487095
ar.L4         2.727917
ma.L1        26.148940
ma.L2        16.804067
ma.L3         9.705584
sigma2       38.546442
dtype: float64

### Specific-To-General Approach
We will now start with an AR(1) and MA(1) and proceed with an StG approach by looking at the t-values of the estimated parameters:

In [15]:
# Estimating an AR(1)
res = tsa.SARIMAX(term, order=(1, 0, 0), trend="c").fit()
res.tvalues

intercept      2.994413
ar.L1        153.773330
sigma2        57.831764
dtype: float64

In [16]:
# Estimating an MA(1)
res = tsa.SARIMAX(term, order=(0, 0, 1), trend="c").fit()
res.tvalues

/Users/talespadilha/opt/anaconda3/envs/python_mfe/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


intercept    24.061317
ma.L1        73.865350
sigma2       21.763449
dtype: float64

In [17]:
# Estimating an AR(2)
res = tsa.SARIMAX(term, order=(2, 0, 0), trend="c").fit()
res.tvalues

intercept     4.136973
ar.L1        92.943054
ar.L2       -22.097041
sigma2       49.974398
dtype: float64

In [18]:
# Estimating an ARMA(1,1)
res = tsa.SARIMAX(term, order=(1, 0, 1), trend="c").fit()
res.tvalues

intercept      3.948588
ar.L1        116.642232
ma.L1         33.614000
sigma2        42.834021
dtype: float64

In [19]:
# Estimating an ARMA(2,1)
res = tsa.SARIMAX(term, order=(2, 0, 1), trend="c").fit()
res.tvalues

intercept     3.644060
ar.L1        18.314437
ar.L2         6.061021
ma.L1        16.702826
sigma2       38.381224
dtype: float64

In [20]:
# Estimating an ARMA(1,2)
res = tsa.SARIMAX(term, order=(1, 0, 2), trend="c").fit()
res.tvalues

intercept      3.615044
ar.L1        116.674005
ma.L1         25.845690
ma.L2         -3.657096
sigma2        37.608982
dtype: float64

In [21]:
# Estimating an ARMA(3,1)
res = tsa.SARIMAX(term, order=(3, 0, 1), trend="c").fit()
res.tvalues

intercept     3.692895
ar.L1        11.741746
ar.L2         4.770352
ar.L3        -1.868836
ma.L1        12.630009
sigma2       37.700351
dtype: float64

In [22]:
# Estimating an ARMA(2,2)
res = tsa.SARIMAX(term, order=(2, 0, 2), trend="c").fit()
res.tvalues

intercept     3.173508
ar.L1         2.130645
ar.L2         3.043435
ma.L1         4.926000
ma.L2         1.637970
sigma2       37.886749
dtype: float64

StG Approach suggests sticking with an ARMA(2,1)!

In [23]:
stg_res = tsa.SARIMAX(term, order=(2, 0, 1), trend="c").fit()
stg_res.tvalues

intercept     3.644060
ar.L1        18.314437
ar.L2         6.061021
ma.L1        16.702826
sigma2       38.381224
dtype: float64

In [1]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)